## OWSlib based birdhouse WCS interaction

In [32]:
endpoint = 'http://localhost:8082/csw'
# connect to CSW, explore it's properties
#endpoint = 'http://www.ngdc.noaa.gov/geoportal/csw' # NGDC Geoportal
#endpoint = 'http://data.nodc.noaa.gov/geoportal/csw'  # NODC Geoportal: collection level
    
#endpoint = 'http://geodiscover.cgdi.ca/wes/serviceManagerCSW/csw'  # NRCAN CUSTOM
#endpoint = 'http://geoport.whoi.edu/gi-cat/services/cswiso' # USGS Woods Hole GI_CAT
#endpoint = 'http://cida.usgs.gov/gdp/geonetwork/srv/en/csw' # USGS CIDA Geonetwork
#endpoint = 'http://www.nodc.noaa.gov/geoportal/csw'   # NODC Geoportal: granule level


In [31]:
from owslib.csw import CatalogueServiceWeb
from owslib import fes
csw = CatalogueServiceWeb('http://localhost:8082/csw')

In [2]:
csw.identification.type
csw.version

'2.0.2'

In [3]:
[op.name for op in csw.operations]

['GetCapabilities',
 'Transaction',
 'GetRepositoryItem',
 'DescribeRecord',
 'GetDomain',
 'GetRecordById',
 'GetRecords',
 'Harvest']

In [4]:
csw.getrecords2()
csw.results

{'matches': 3, 'nextrecord': 0, 'returned': 3}

In [5]:
from owslib.fes import PropertyIsEqualTo, PropertyIsLike, BBox

In [6]:
csw.getrecords2(maxrecords=5)
csw.results

{'matches': 3, 'nextrecord': 0, 'returned': 3}

In [18]:
test_query = PropertyIsEqualTo('csw:AnyText', 'test')
csw.getrecords2(constraints=[test_query], maxrecords=20)
csw.results

{'matches': 3, 'nextrecord': 0, 'returned': 3}

In [35]:
def displayrecords(url):
    try:
        csw = CatalogueServiceWeb(url)
        csw.getrecords2()
        csw.results
        for rec in csw.records:
            print csw.records[rec].title
    except:
        print 'something went wrong'
 

In [36]:
displayrecords(endpoint)

Malleefowl
testpub1
testpub2


In [46]:
for rec,item in csw.records.iteritems():
    print rec
    print item.abstract
    print item.modified
    print item.subjects
    print item.title
    print item.format
    print item.creator
    print item.type
    print '-----------------------------------------'

urn:uuid:a6e37330-817b-45ca-aaaf-62d7acf51f11
Malleefowl Processes (esgf, workflow, publish, security, ...)
None
['WPS', 'PyWPS', 'Birdhouse', 'malleefowl']
Malleefowl
WPS
None
service
-----------------------------------------
urn:uuid:f3a84f97-ae8f-48f7-a13a-ac87eb0baf3e
test publication 1 dummy
2015-10-07T13:37:35Z
['test', 'data']
testpub1
txt
None
dataset
-----------------------------------------
urn:uuid:e69f2b39-b783-45d8-9db8-536a47909fae
test publication 1 dummy + type
2015-10-07T14:30:25Z
['test', 'service']
testpub2
ws
None
service
-----------------------------------------


In [41]:
print(csw.records.keys())

['urn:uuid:a6e37330-817b-45ca-aaaf-62d7acf51f11', 'urn:uuid:f3a84f97-ae8f-48f7-a13a-ac87eb0baf3e', 'urn:uuid:e69f2b39-b783-45d8-9db8-536a47909fae']


In [22]:

# changed template to use "type" element
#
#csw.transaction(ttype="insert", typename='csw:Record', record=str(record))

from mako.template import Template
import uuid
def publish(my_csw, record):
    from os.path import join, dirname

    record['identifier'] = uuid.uuid4().get_urn()
    templ_dc = Template(filename="dublin_core.xml")
    my_csw.transaction(ttype="insert", typename='csw:Record', record=str(templ_dc.render(**record)))
    
record = {}
record["keywords"]='test,service'
record["abstract"]='test publication 1 dummy + type'
record['title']='testpub2'
record['format']='ws'
record['creator']='Stephan Kindermann'
record['source']='manual ingest'
record['rights']='open'
record['type']='service'

In [18]:
print record

{'rights': 'open', 'format': 'txt', 'abstract': 'test publication 1 dummy', 'creator': 'Stephan Kindermann', 'source': 'manual ingest', 'keywords': 'test,data', 'title': 'testpub1'}


In [23]:
publish(csw,record)

In [47]:
csw.getrecords2(cql='csw:AnyText like "%testtt%"')
csw.results

{'matches': 0, 'nextrecord': 0, 'returned': 0}

Links
* https://github.com/rsignell-usgs
* http://nbviewer.ipython.org/github/nccoos/ioos-vocabulary-search/blob/master/examples/example_csw_ngdc_geoportal.ipynb
* https://www.wakari.io/sharing/bundle/rsignell/NGDC-CSW-DAP
* https://www.wakari.io/sharing/bundle/rsignell/DATA.NOAA.GOV-CSW-DAP
* https://gist.github.com/epifanio/5452800
* https://geopython.github.io/pycsw-workshop/docs/advanced/advanced-exercises.html